# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import os

import gmaps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv(output_data_file)

cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ushuaia,40,AR,17-06-2020,69,-54.80,-68.30,39.20,0.89
1,Balakovo,0,RU,17-06-2020,78,52.03,47.80,68.11,4.97
2,Carnarvon,40,AU,17-06-2020,100,-24.87,113.63,68.00,5.82
3,Butaritari,20,KI,17-06-2020,77,3.07,172.79,82.47,14.52
4,Pathein,100,MM,17-06-2020,89,16.78,94.73,77.36,4.16


In [3]:
cities_df.dtypes

City           object
Cloudiness      int64
Country        object
Date           object
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Data for gmaps.
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]
# Value for max intensity.
maxHumidity = cities_df["Humidity"].max()

In [5]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity,
    dissipating=False,
    max_intensity=maxHumidity,  # Scales based on the max humidity of the dataset..
    point_radius=1,
)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down cities to fir our ideal conditions.
hotel_df = cities_df[
    (cities_df["Max Temp"] >= 80)
    & (cities_df["Max Temp"] <= 90)
    & (cities_df["Cloudiness"] <= 10)
    & (cities_df["Wind Speed"] <= 10)
]

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
25,Kerman,0,IR,17-06-2020,10,30.28,57.08,86.00,6.93
35,Half Moon Bay,5,US,17-06-2020,27,37.46,-122.43,80.01,6.93
42,Albany,0,US,17-06-2020,29,42.60,-73.97,86.00,2.71
80,Cabo San Lucas,5,MX,17-06-2020,52,22.89,-109.91,89.60,3.36
99,Trairi,0,BR,17-06-2020,81,-3.28,-39.27,81.50,9.93
135,Morro Bay,0,US,17-06-2020,40,35.37,-120.85,81.00,3.00
151,São Miguel do Araguaia,0,BR,17-06-2020,39,-13.28,-50.16,85.33,8.08
166,Kiryat Gat,0,IL,17-06-2020,44,31.61,34.76,80.01,4.14
185,Saint George,1,US,17-06-2020,26,37.10,-113.58,80.60,8.05
196,Bethel,1,US,17-06-2020,50,41.37,-73.41,82.00,6.55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create empty column to store hotel names
hotel_df["Hotel Name"] = ""

# Filter the columns show only the info we want.
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Hotel Name"]]

hotel_df

/Users/Sebastian/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Lat,Lng,Hotel Name
25,Kerman,IR,30.28,57.08,
35,Half Moon Bay,US,37.46,-122.43,
42,Albany,US,42.60,-73.97,
80,Cabo San Lucas,MX,22.89,-109.91,
99,Trairi,BR,-3.28,-39.27,
135,Morro Bay,US,35.37,-120.85,
151,São Miguel do Araguaia,BR,-13.28,-50.16,
166,Kiryat Gat,IL,31.61,34.76,
185,Saint George,US,37.10,-113.58,
196,Bethel,US,41.37,-73.41,


In [8]:
# Set up the base url for the google nearby search.

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # Set up search parameters for 5km.
    params = {
        "location": f"{row['Lat']},{row['Lng']}",
        "radius": 50000,  # We are not using rank ny distance because we want to rank by importance thus we leave default and use radius only
        "type": "lodging",
        "key": g_key,
    }

    # Response
    response = requests.get(base_url, params=params).json()

    # extract results
    results = response["results"]

    try:
        # Stores the hotel name under the hotel column.
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]

    # If the search cannot find a hotel in 5km radius, try another search for a 10km radius.
    except (KeyError, IndexError):
        try:
            params = {
                "location": f"{row['Lat']},{row['Lng']}",
                "radius": 10000,
                "type": "lodging",
                "key": g_key,
            }
            response = requests.get(base_url, params=params).json()

            # extract results
            results = response["results"]

            hotel_df.loc[index, "Hotel Name"] = results[0]["name"]

        # If a hotel is not found in a 10km radius, skip the search.
        except (KeyError, IndexError):
            print(f"Missing field/result... skipping. {row['City']}")

/Users/Sebastian/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [9]:
# Removes cities without hotels in the desired radius range.
narrowed_city_df = hotel_df[hotel_df["Hotel Name"] != ""]

narrowed_city_df

,City,Country,Lat,Lng,Hotel Name
25,Kerman,IR,30.28,57.08,Kerman Tourism Hotel
35,Half Moon Bay,US,37.46,-122.43,The Clift Royal Sonesta Hotel
42,Albany,US,42.60,-73.97,CrestHill Suites SUNY University Albany
80,Cabo San Lucas,MX,22.89,-109.91,Hotel Tesoro Los Cabos
99,Trairi,BR,-3.28,-39.27,Hotel VentoBrasil
135,Morro Bay,US,35.37,-120.85,Inn at Morro Bay
151,São Miguel do Araguaia,BR,-13.28,-50.16,Pousada Canoeiros
166,Kiryat Gat,IL,31.61,34.76,West All Suite Hotel Ashdod - מלון ווסט אשדוד
185,Saint George,US,37.10,-113.58,Best Western Plus Abbey Inn
196,Bethel,US,41.37,-73.41,Courtyard by Marriott Danbury


In [10]:
# Templeate for map tags.
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Gets the info for each hotel.
hotel_info = [
    info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()
]
locations = narrowed_city_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))